In [19]:
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
#lei=law, n=article number, p=paragraph number, inciso=item, alinea=point
class Scrap:
    def __init__(self, lei, n=None, p=None, inciso=None):
        """Initializing the driver"""
        self.lei = lei
        self.n = n
        self.p = p
        self.inciso1 = inciso
        #self.alinea1 = alinea
    
        self.art = str('Art. ' + str(self.n))
        self.lista_paragrafo = []
        self.lista_inciso = []
        
        #constantes auxiliares
        self.z = 0
        
        url = 'https://www2.camara.leg.br/busca/?q=' + lei
        option = Options()
        option.headless = True
        driver = webdriver.Firefox(options=option)
        driver.get(url)
        driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
        driver.find_element_by_xpath("//div[@class='sessao']//a").click()
        element = driver.find_element_by_xpath("//div[@class='texto']")
        self.titulo = driver.find_element_by_class_name('ementa')
        html_content = element.get_attribute('outerHTML')
        self.titulo = self.titulo.get_attribute('outerHTML')
        #time.sleep(2)
        driver.quit()

        self.soup = BeautifulSoup(html_content, 'lxml')
        self.texto = self.soup.find_all(text=True)

        self.titulo = BeautifulSoup(self.titulo, 'lxml')
        self.titulo = self.titulo.find_all(text=True)
        #limpando
        for i in range(len(self.texto)):
            self.texto[i] = self.texto[i].replace('\n', '')
            self.texto[i] = self.texto[i].replace('\t', '')
            self.texto[i] = self.texto[i].replace('\xa0', '')
        
        
    def title(self):
        """Returns the law title""" 
        for i in range(len(self.titulo)):
            self.titulo[i] = self.titulo[i].replace('\n', '')
            self.titulo[i] = self.titulo[i].replace('\t', '')
            if '' in self.titulo:
                self.titulo.remove('')
            return(self.titulo[0])
   
    def article(self):
        pattern0 = f'(Art\. {self.n})'
        patternf = r"(\'\,)"
        self.artigo0 = re.search(pattern0, str(self.texto)).start()
        self.artigof = re.search(patternf, str(self.texto)[self.artigo0:]).start()
        self.total = self.artigo0 + self.artigof
        return(str(self.texto)[self.artigo0:(self.total)])

    def paragraph(self):
        """Must returns the requested paragraph for requested article"""
        pattern0 = f'(Art\. {self.n})'
        patternf = r"(\'\,)"
        artigo0 = re.search(pattern0, str(self.texto)).start()
        artigof = re.search(patternf, str(self.texto)[artigo0:]).start()

        pattern_paragraph0 = f'(§ {self.p})'
        pattern_paragraphf = r"(\'\,)"
        paragraph0 = re.search(pattern_paragraph0, str(self.texto)[artigof:]).start()
        paragraph0 = paragraph0 + artigof
        paragraphf = re.search(pattern_paragraphf, str(self.texto)[paragraph0:]).start()
        paragraphf = paragraphf + paragraph0
        try:
            if (f'Art. {int(self.n) +1}') in str(self.texto)[(artigo0 + artigof): paragraphf].split("',")[self.p]:
                return('Parágrafo não encontrado!')
            else:
                return(str(self.texto)[(artigo0 + artigof): paragraphf].split("',"))[self.p]
        except:
            if (f'Art. {int(self.n) +1}') in str(self.texto)[(artigo0 + artigof): paragraphf].split("',")[self.p]:
                return('Parágrafo não encontrado!')
            else:
                return(str(self.texto)[(artigo0 + artigof):].split("',"))[self.p]
    
    
    def inciso(self):
        """Returns the requested item from the article / law, 
        if no item was requested the entire list is returned"""
        self.article()
        if self.inciso1 is not None:
            for i in range(self.i2, len(self.texto)):
                if (f'Art. {int(self.n) +1}') in self.texto[i]:
                    break
                if (str(self.inciso1)) in self.texto[i]:
                    self.texto[i] = self.texto[i].replace('\xa0', '')
                    return(self.texto[i])
            return('O inciso requisitado não foi encontrado!')
        else:
            for i in range(self.i2, len(self.texto)):
                if 'SEÇÃO' in self.texto[i]:
                    if(len(self.lista_inciso) == 0):
                        return('O artigo requisitado não possui incisos!')
                    else: 
                        return(self.lista_inciso)
                    
                if (f'Art. {int(self.n) +1}') in self.texto[i]:
                    if(len(self.lista_inciso) == 0):
                        return('O artigo requisitado não possui incisos!')
                    else: 
                        return(self.lista_inciso)
                
                if ('I') in self.texto[i]:
                    for j in range(i, len(self.texto)):
                        if (f'Art. {int(self.n) +1}') in self.texto[j]:
                            if(len(self.lista_inciso) == 0):
                                return('O artigo requisitado não possui incisos!')
                            else: 
                                return(self.lista_inciso)
                        if ('\n') in self.texto[j]:
                            return(self.lista_inciso)
                        self.texto[j] = self.texto[j].replace('\xa0', '')
                        self.lista_inciso.append(self.texto[j])
            if(len(self.lista_inciso) == 0):
                return('O artigo requisitado não possui incisos!')
            else:
                return(self.lista_inciso)

In [ ]:
teste = Scrap(lei='lei 1.060', n=4, p=1)

In [ ]:
teste.title()

In [ ]:
teste.article()

In [ ]:
teste.paragraph()

In [177]:
#teste.inciso()

In [112]:
#outra instância
teste2 = Scrap(lei='lei 1.060', n=7)

In [113]:
teste2.title()

'Estabelece normas para a concessão de assistência judiciária aos necessitados. '

In [114]:
teste2.article()

'Art. 7ºA parte contrária poderá, em qualquer fase da lide, requerer a revogação dos benefícios de assistência, desde que prove a inexistência ou o desaparecimento dos requisitos essenciais à sua concessão. '

In [115]:
teste2.paragraph()

('Art. 7ºA parte contrária poderá, em qualquer fase da lide, requerer a revogação dos benefícios de assistência, desde que prove a inexistência ou o desaparecimento dos requisitos essenciais à sua concessão. ',
 ['Tal requerimento não suspenderá o curso da ação e se processará pela forma estabelecida no final do artigo 6º desta Lei. '])

In [116]:
teste2.inciso()

'O artigo requisitado não possui incisos!'

In [117]:
#outra instância
teste3 = Scrap(lei ='lei 9503', n=6)

In [118]:
teste3.title()

'Institui o Código de Trânsito Brasileiro. '

In [119]:
teste3.article()

'Art. 6º. São objetivos básicos do Sistema Nacional de Trânsito: '

In [120]:
teste3.paragraph()

'O artigo requisitado não possui parágrafos.'

In [121]:
teste3.inciso()

['I - estabelecer diretrizes da Política Nacional de Trânsito, com vistas à segurança, à fluidez, ao conforto, à defesa ambiental e à educação para o trânsito, e fiscalizar seu cumprimento; ',
 'II - fixar, mediante normas e procedimentos, a padronização de critérios técnicos, financeiros e administrativos para a execução das atividades de trânsito; ',
 'III - estabelecer a sistemática de fluxos permanentes de informações entre os seus diversos órgãos e entidades, a fim de facilitar o processo decisório e a integração do Sistema. ']

In [145]:
#outra instancia
teste4 = Scrap(lei ='lei 9503', n=7)

In [146]:
teste4.title()

'Institui o Código de Trânsito Brasileiro. '

In [147]:
teste4.article()

'Art. 7º. Compõem o Sistema Nacional de Trânsito os seguintes órgãos e entidades: '

In [148]:
teste4.paragraph()

'O artigo requisitado não possui parágrafos.'

In [149]:
teste4.inciso()

['I - o Conselho Nacional de Trânsito - CONTRAN, coordenador do Sistema e órgão máximo normativo e consultivo; ',
 'II - os Conselhos Estaduais de Trânsito - CETRAN e o Conselho de Trânsito do Distrito Federal - CONTRANDIFE, órgãos normativos, consultivos e coordenadores; ',
 'III - os órgãos e entidades executivos de trânsito da União, dos Estados, do Distrito Federal e dos Municípios; ',
 'IV - os órgãos e entidades executivos rodoviários da União, dos Estados, do Distrito Federal e dos Municípios; ',
 'V - a Policia Rodoviária Federal; ',
 ' VI - as Polícias Militares dos Estados e do Distrito Federal; e ',
 'VII - as Juntas Administrativas de Recursos de Infrações - JARI. ']

In [127]:
teste5 = Scrap(lei = 'lei 1060', n = 13)

In [128]:
teste5.title()

'Aprova o texto do Decreto-Lei nº 1104, de 30 de abril de 1970, que altera o Decreto-Lei nº 1060, de 21 de outubro de 1969.'

In [129]:
teste5.article()

In [130]:
teste5.paragraph()

AttributeError: 'Scrap' object has no attribute 'i2'